In [1]:
import pandas as pd
import smtplib, ssl
ssl._create_default_https_context = ssl._create_unverified_context
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import json
import requests
!sqlite3 TBD307.db < TBD307.sql
import sqlite3
from datetime import date

In [2]:
# cycle through days
date = date.today()
# get weather info 
f = r"http://api.openweathermap.org/data/2.5/weather?q=Ames&appid=32d018663eb19deb360c0548ce6e4e46"
response = requests.get(f)
data_struct = json.loads(response.text)
temp_f = (data_struct['main']['temp'] - 273.15) * 9/5 + 32
temp_f = round(temp_f)
condition = data_struct['weather'][0]['main']

# store info into df

In [3]:
connection = sqlite3.connect('TBD307.db')

In [4]:
connection.execute(f"""INSERT INTO weather (date, temp, condition) VALUES ('{date}','{temp_f}','{condition}')""")
df = pd.read_sql('SELECT * FROM weather', connection)

In [7]:
df

,date,temp,condition
0,2022-04-28,62,Clear


In [6]:
port = 465  # For SSL
#smtp_server = "smtp.gmail.com"
smtpserver = smtplib.SMTP("smtp.gmail.com",587)
sender_email = "tbdproject307@gmail.com"  # Enter your address
receiver_email = "mcelmee20@gmail.com"  # Enter receiver address
password = 'testing307!'
message = MIMEMultipart("alternative")
message["Subject"] = "weather test"
#smtpserver.ehlo()
#smtpserver.starttls()
#smtpserver.ehlo() # extra characters to permit edit
#smtpserver.login(sender_email, password)
text = """\
AND TODAYS WEATHER ISSSSSSSS:"""
html = f'{df}'
# Turn these into plain/html MIMEText objects
part1 = MIMEText(text, "plain")
part2 = MIMEText(html, "html")

# Add HTML/plain-text parts to MIMEMultipart message
# The email client will try to render the last part first
message.attach(part1)
message.attach(part2)


context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(
        sender_email, receiver_email, message.as_string()
    )